In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ..

In [ ]:
import os

import numpy as np
import torch
from torch.nn import functional as F
import matplotlib.pyplot as plt
import IPython.display as ipd
import wandb

from deepspeech import model, train, datasets, utils

# YesNo

In [ ]:
p = model.HParams(graphemes=datasets.YESNO_GRAPHEMES)
ds = datasets.SpecAugmented(datasets.yesno(p), p, masked=True)

In [ ]:
def display(ds, i):
    x, y = ds[i]
    xin, _, yin = ds.data.data[i]
    ipd.display(ipd.Audio(xin, rate=p.sampling_rate))
    print(yin, y)
    plt.figure(figsize=(20, 20))
    plt.imshow(x.log2().detach().numpy())
    plt.plot()

In [ ]:
display(ds, 0)

In [ ]:
display(ds, 1)

In [ ]:
display(ds, 2)

In [ ]:
display(ds, 3)

## Collation

In [ ]:
batch_size = 5
batch = [ds[i] for i in range(batch_size)]
x, nx, y, ny = datasets.batch(p)(batch)
nx_dowsampled = p.frame_lengths(nx)

In [ ]:
nx, ny

In [ ]:
y

In [ ]:
for i in range(batch_size):
    print('want: ', ds[i][1])
    print('gots: ', ''.join([p.graphemes[int(c)] for c in y[i]]))
    print()

In [ ]:
for i in range(batch_size):
    print('melgram: ', ds[i][0].shape)
    print('audio: ', ds.data.data[i][0].shape)
    print('batch: ', x[i].shape)
    print('nx: ', nx[i])
    print('nx_dowsampled: ', nx_dowsampled[i])
    print('ny: ', ny[i])
    print()

In [ ]:
for i in range(batch_size):
    plt.figure(figsize=(20, 20))
    plt.imshow(x[i].log2().detach().numpy())